In [1]:
import os
import json

# DPO in Training

In [2]:
DIR = "../eval_dpo_training"

files = os.listdir(DIR)
temperatures = [0.7, 1.0]
data_dict = {}

for temperature in temperatures:
    steps = []
    winrates = []
    for file in files:
        # Ignore non-json files
        if not file.endswith(".json"):
            continue

        # Get the temperature from the file name, e.g. eval_dpo_step_100_temp_0.7_20250823_172153_20251024_100637.json
        temp = float(file.split("_")[5])
        if temp == temperature:
            # Get step from file name, e.g. eval_dpo_step_100_temp_0.7_20250823_172153_20251024_100637.json
            step = int(file.split("_")[3])
            steps.append(step)

            # Load the json file
            with open(os.path.join(DIR, file), "r") as f:
                data = json.load(f)
                
            # Load component data
            metadata = data["metadata"]
            winrate = data["winrates"]["model_winrate"]
            winrates.append(winrate)
    
    # Sort steps and winrates by steps
    steps, winrates = zip(*sorted(zip(steps, winrates)))

    data_dict[temperature] = {
        "steps": steps,
        "winrates": winrates
    }


print(data_dict)

{0.7: {'steps': (100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000), 'winrates': (40.666666666666664, 42.66666666666667, 44.666666666666664, 44.0, 40.0, 42.66666666666667, 51.33333333333333, 46.0, 48.66666666666667, 48.66666666666667, 56.666666666666664, 50.0, 52.666666666666664, 54.666666666666664, 56.666666666666664, 57.333333333333336, 56.666666666666664)}, 1.0: {'steps': (100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, 8000), 'winrates': (46.666666666666664, 56.666666666666664, 53.333333333333336, 54.0, 51.33333333333333, 55.333333333333336, 51.33333333333333, 46.0, 51.33333333333333, 52.0, 50.66666666666667, 49.333333333333336, 52.666666666666664, 58.666666666666664, 51.33333333333333, 51.33333333333333, 53.333333333333336)}}


In [3]:
import plotly.graph_objects as go
import plotly.express as px

# Create figure
fig = go.Figure()

data = data_dict
# Add traces for each temperature
for temperature, values in data.items():
    fig.add_trace(go.Scatter(
        x=values['steps'],
        y=values['winrates'],
        mode='lines+markers',
        name=f'Temperature {temperature}',
        line=dict(width=2),
        marker=dict(size=8)
    ))

# Update layout
fig.update_layout(
    title='Model Winrate vs Training Steps',
    xaxis_title='Training Steps',
    yaxis_title='Winrate (%)',
    legend_title='Temperature',
    hovermode='x unified',
    template='plotly_white',
    width=1000,
    height=600,
    font=dict(size=12)
)

# Add grid
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

# Show the plot
fig.show()

# Save the plot to pdf
# os.makedirs("plots", exist_ok=True)
# fig.write_pdf(file=f'plots/model_winrate_vs_training_steps.pdf')

# Baselines

In [4]:
DIR = "../eval_baselines"

files = os.listdir(DIR)

data_dict = {}

for file in files:
    if not file.endswith(".json"):
        continue

    # Get name from file, e.g. eval_pythia-2.8b-base-0.5_20251024_092244_20251024_121009.json -> pythia-2.8b-base
    name = file.split("_")[1]

    # Open the json file
    with open(os.path.join(DIR, file), "r") as f:
        data = json.load(f)

    metadata = data["metadata"]
    temperature = metadata["temperature"]
    winrate = data["winrates"]["model_winrate"]
    
    data_dict[name] = {
        "temperature": temperature,
        "winrate": winrate
    }

print(data_dict)

    

{'pythia-2.8b-sft-1.0': {'temperature': 1.0, 'winrate': 46.0}, 'pythia-2.8b-base-0.25': {'temperature': 0.25, 'winrate': 22.0}, 'pythia-2.8b-sft-0.5': {'temperature': 0.5, 'winrate': 34.0}, 'pythia-2.8b-base-1.0': {'temperature': 1.0, 'winrate': 29.333333333333332}, 'pythia-2.8b-sft-0.75': {'temperature': 0.75, 'winrate': 46.666666666666664}, 'pythia-2.8b-dpo-0.25': {'temperature': 0.25, 'winrate': 25.333333333333336}, 'pythia-2.8b-base-0.5': {'temperature': 0.5, 'winrate': 27.333333333333332}, 'pythia-2.8b-sft-0.25': {'temperature': 0.25, 'winrate': 29.333333333333332}, 'pythia-2.8b-base-0.75': {'temperature': 0.75, 'winrate': 29.333333333333332}, 'pythia-2.8b-dpo-1.0': {'temperature': 1.0, 'winrate': 57.99999999999999}, 'pythia-2.8b-dpo-0.5': {'temperature': 0.5, 'winrate': 64.0}, 'pythia-2.8b-dpo-0.75': {'temperature': 0.75, 'winrate': 52.666666666666664}}


In [6]:
from collections import defaultdict

data = data_dict
# Organize data by model type
model_data = defaultdict(lambda: {'temperatures': [], 'winrates': []})

# Model name mapping
model_names = {
    'base': 'Pythia-2.8B (Base)',
    'sft': 'Preferred-FT (SFT)',
    'dpo': 'DPO'
}

# Parse the data
for key, values in data.items():
    # Extract model type from key
    if 'base' in key:
        model_type = 'base'
    elif 'dpo' in key:
        model_type = 'dpo'
    elif 'sft' in key:
        model_type = 'sft'
    else:
        continue
    
    model_data[model_type]['temperatures'].append(values['temperature'])
    model_data[model_type]['winrates'].append(values['winrate'])

# Sort by temperature for each model
for model_type in model_data:
    temps = model_data[model_type]['temperatures']
    wins = model_data[model_type]['winrates']
    # Sort both lists by temperature
    sorted_pairs = sorted(zip(temps, wins))
    model_data[model_type]['temperatures'] = [t for t, w in sorted_pairs]
    model_data[model_type]['winrates'] = [w for t, w in sorted_pairs]

# Create figure
fig = go.Figure()

# Color scheme
colors = {
    'base': '#636EFA',  # Blue
    'sft': '#EF553B',   # Red
    'dpo': '#00CC96'    # Green
}

# Add traces for each model
for model_type in ['base', 'sft', 'dpo']:
    if model_type in model_data:
        fig.add_trace(go.Scatter(
            x=model_data[model_type]['temperatures'],
            y=model_data[model_type]['winrates'],
            mode='lines+markers',
            name=model_names[model_type],
            line=dict(width=3, color=colors[model_type]),
            marker=dict(size=12, line=dict(width=2, color='white')),
            hovertemplate='<b>Temperature:</b> %{x}<br><b>Winrate:</b> %{y:.2f}%<extra></extra>'
        ))

# Update layout
fig.update_layout(
    title=dict(
        text='Model Winrate vs Temperature',
        x=0.5,
        xanchor='center',
        font=dict(size=20, family='Arial Black')
    ),
    xaxis=dict(
        title='Temperature',
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgray',
        dtick=0.25  # Show ticks at 0.25, 0.5, 0.75, 1.0
    ),
    yaxis=dict(
        title='Winrate (%)',
        showgrid=True,
        gridwidth=1,
        gridcolor='lightgray',
        range=[10, 70]  # Adjust based on your data
    ),
    legend=dict(
        title='Model',
        orientation='v',
        yanchor='top',
        y=0.98,
        xanchor='right',
        x=0.98,
        font=dict(size=12),
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='lightgray',
        borderwidth=1
    ),
    hovermode='x unified',
    template='plotly_white',
    width=1000,
    height=600,
    font=dict(size=12)
)

# Show plot
fig.show()

# Optional: Save to HTML
# fig.write_html("model_winrate_comparison.html")
# print("Plot saved to: model_winrate_comparison.html")

# Table

In [7]:
from collections import defaultdict

# Organize data by model and temperature
model_data = defaultdict(dict)
temperatures = set()

for key, values in data.items():
    # Extract model type
    if 'base' in key:
        model_type = 'Base'
    elif 'dpo' in key:
        model_type = 'DPO'
    elif 'sft' in key:
        model_type = 'SFT'
    else:
        continue
    
    temp = values['temperature']
    winrate = values['winrate']
    
    model_data[model_type][temp] = winrate
    temperatures.add(temp)

# Sort temperatures
temperatures = sorted(temperatures)

# Model order
model_order = ['Base', 'SFT', 'DPO']

# Generate LaTeX table
latex_code = r"""\begin{table}[htbp]
\centering
\caption{Model Winrate (\%) across Different Temperatures}
\label{tab:winrate}
\begin{tabular}{l|cccc}
\toprule
\textbf{Model} & \textbf{T=0.25} & \textbf{T=0.5} & \textbf{T=0.75} & \textbf{T=1.0} \\
\midrule
"""

for model in model_order:
    if model not in model_data:
        continue
    
    row = f"{model}"
    for temp in temperatures:
        if temp in model_data[model]:
            winrate = model_data[model][temp]
            row += f" & {winrate:.2f}"
        else:
            row += " & --"
    row += " \\\\\n"
    latex_code += row

latex_code += r"""\bottomrule
\end{tabular}
\end{table}
"""

print(latex_code)

# Save to file
# with open("winrate_table.tex", "w") as f:
#     f.write(latex_code)

# print("\nLaTeX table saved to: winrate_table.tex")

\begin{table}[htbp]
\centering
\caption{Model Winrate (\%) across Different Temperatures}
\label{tab:winrate}
\begin{tabular}{l|cccc}
\toprule
\textbf{Model} & \textbf{T=0.25} & \textbf{T=0.5} & \textbf{T=0.75} & \textbf{T=1.0} \\
\midrule
Base & 22.00 & 27.33 & 29.33 & 29.33 \\
SFT & 29.33 & 34.00 & 46.67 & 46.00 \\
DPO & 25.33 & 64.00 & 52.67 & 58.00 \\
\bottomrule
\end{tabular}
\end{table}

